# Bias - Variance Tradeoff 

We beginnen met de initialisatie:

In [3]:
!pip install numpy scikit-learn
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 8.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 7.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 7.1 MB/s eta 0:00:0000:0100:01


In dit workbook gaan we het hebben over de 3 verschillende oorzaken waarom een voorspellend model niet perfect voorspelt op een test set. De 3 redenen zijn:
1. de irreducibele fout,
2. de bias,
3. de variance.

De irreducibele fout is lastig te voorkomen. Als we een targetvariabele $y$ voorspellen op basis van $x$, dan kan het zijn dat er nog veel andere factoren zijn die $y$ ook beïnvloeden, maar waar we geen data van hebben. 
Als we bijvoorbeeld het inkomen voorspellen op basis van het opleidingsniveau, krijgen we nooit een perfecte voorspelling, omdat bijvoorbeeld leeftijd ook een rol speelt. 
Dit wordt meestal al duidelijk in de data set: er komen twee personen voor met hetzelfde opleidingsniveau en toch hebben beiden een ander inkomen. Dit heet de irreducibele fout. De enige manier om de irreducibele fout 
kleiner te maken is om meer voorspellende variabelen mee te nemen, maar dan moet je daar dus ook data van hebben. 

De bias is een afwijking in de voorspelling, omdat het gekozen model te eenvoudig is om het werkelijke verband tussen $x$ en $y$ precies te beschrijven. Als je bijvoorbeeld met lineaire regressie de tijd wilt voorspellen 
dat een voorwerp nodig heeft om van een hooge $h$ op de vloer te vallen, dan gaat dat niet precies lukken. Dit omdat het werkelijke verband tussen valtijd en hoogte een tweede orde polynoom is, en dus niet met een lineaire functie 
precies beschreven kan worden. Een iets complexer model zal het hier waarschijnlijk beter doen.

De variance is de ingewikkelste oorzaak van een niet-ideale voorspelling op de test set. De achterliggende oorzaak is dat een model getraind wordt op basis van een train set. Maar deze train set is meestal niet volkomen 
representatief. Vaak is het een steekproef vanuit een grotere populatie. Als een tweede keer een steekproef zou worden getrokken zouden we net andere observaties kunnen krijgen en dus ook andere geleerde parameters in ons model.
Soms bestaat de train set zelfs uit de gelabelde data die toevallig beschikbaar zijn. Zelfs in het geval dat de train en test set geconstrueerd zijn uit dezelfde ruwe data set door middel van het random toewijzen aan de 
train set of de test set, dan nog is er een toevalselement aanwezig. Bij herhaling kan dit leiden tot een andere train set en dus andere geleerde parameters. Indien er veel data beschikbaar is en een model met weinig parameters
is de variance meestal geen groot probleem, maar dat wordt anders bij modellen met veel parameters en / of miner data.
  
De take-away van dit notebook is dat bias en variance zich vaak gedragen als een wip: zodra je de bias omlaag probeert te krijgen door een complexer model te kiezen, gaat de variance omhoog. En zodra je de variance verlaagt 
met een eenvoudiger model, gaat de bias weer omhoog. Het gaat erom een balans te vinden en te weten dat je niet altijd moet kiezen voor het meest ingewikkelde model. Alleen met meer data kun je meestal beide naar beneden krijgen. Maar voordat we het gaan hebben over bias en variance, eerst wat nuttige kennis over supervised learning algoritmen in het algemeen!

<div style="background-color: lightyellow">


# Opdracht
Vraag aan chatGPT, claude of een ander Large Language Model om een uitleg te geven over de bias-variance tradeoff. Vraag ook of het model een stukje code kan schrijven om het te verduidelijken.
</div>

## Redeneren over álle supervised learning algoritmen
We kijken in dit notebook naar supervised learning algoritmen in het algemeen. Dus niet specifiek naar lineaire regressie, logistische regressie, decision trees, random forest, XGBoost, neurale netwerken, of een ander specifiek algoritme. Nee, we kijken algemener naar wat al deze algoritmes gezamenlijk hebben. Wat willen deze algoritmen bereiken? En welke elementen hebben ze allemaal met elkaar gemeen? Deze manier van kijken gaat ons natuurlijk geen praktische kennis opleveren over een specifiek algoritme. Maar we zullen zien dat door in het algemeen te kijken naar wat deze groep algoritmen doet, we toch een stukje nuttige kennis kunnen afleiden. In dit geval over de bias-variance trade-off. En het leuke van deze kennis is dat het kennis is die voor alle supeervised learning algoritmen geldt. Dus ook als over 20 jaar volkomen nieuwe algoritmen uitgevonden zijn, dan nog blijft deze kennis geldig en waardevol. Het zal je ook een betere data-scientist maken als je het aspect van bias-variance trade-off kent en in je achterhoofd houdt bij het kiezen van een model. Wel is dit onderdeel een moeilijk onderdeel. Want het abstract redeneren is wat lastiger dan het concreet kunnen vertellen over een specifiek algoritmen. Maar laten we van start gaan!

## Het eerste element dat vaak (impliciet) voorkomt: de structuurfunctie
De meeste supervised learning algoritmen proberen een relatie te vinden tussen een aantal verklarende variabelen $x_1$, $x_2$, ... , $x_n$ en een targetvariabele $y$. De verklarende variabelen zijn meestal getallen en we houden ervan om ze te schrijven als een vector. Vanaf nu zal ik dus ook de vetgedrukte x gebruiken $\mathbf{x}$. Als ik dat doe, dan bedoel ik dus de vector met als eerste element $x_1$ en als laatste element $x_n$: $\mathbf{x}=(x_1,\ldots,x_n)$. Jullie weten dat we de relatie tussen $\mathbf{x}$ en $y$ kunnen schrijven als een functie. In dit geval:

$$
y = f(\mathbf{x}) \tag{1}
$$

Ik noem deze functie in dit notebook de **structuurfunctie** van het supervised learning algoritme. Voor sommige algoritmen zoals lineaire regressie en logistische regressie is de structuurfunctie makkelijk op te schrijven. Voor lineaire regressie geldt bijvoorbeeld:

$$
f(\mathbf{x}) = c_0 + c_1 \cdot x_1 + \ldots + c_n \cdot x_n \tag{2}
$$

Hierbij zijn $c_0, c_1, \ldots, c_n$ getallen die we *parameters* noemen. Deze moeten geleerd worden uit de data. De parameters kunnen we ook als vector schrijven: $\mathbf{c}=c_0, \ldots. c_n$. Maken we nu een nieuwe vector $\mathbf{x'}$ door aan de vector $\mathbf{x}$ een extra element met waarde 1 toe te voegen aan het begin, dan kunnen we formule (2) ook korter schrijven als het inproduct van twee vectoren:

$$
f(\mathbf{x'}) = \mathbf{c} \cdot \mathbf{x'} \tag{3}
$$

Meestal is het lastiger om de structuurfunctie op te schrijven. Voor een beslisboom met 5 binaire splits geldt bijvoorbeeld (zie _Hastie, Tibshirani, Friedman: The elements of Statistical Learning (2001) p. 267_):

$$
g(\mathbf{x})=\sum_{m=1}^5 c_m I\{\mathbf{x} \in R_m\} \tag{4}
$$

Het gaat er ons op dit moment niet om om de structuurfunctie te kunnen opschrijven, het gaat er nu om dat voor heel veel supervised learning algoritmen er zo'n structuurfunctie bestaat. Ook al wordt ie vaak niet opgeschreven omdat het nogal ingewikkeld is. Het bestaan van een onderliggende structuurfunctie is een element dat veel supervised learning algoritmen gemeenschappelijk hebben.

Eigenlijk zijn de formules (1) tot en met (4) hierboven nog niet compleet. In de praktijk gebeurt het nooit dat de targetvariabele $y$ precies voldoet aan de structuurfunctie. Dus dat $y$ bijvoorbeeld exact voorspelt kan worden met bijvoorbeeld formule (2). Daarom voegen we bij al deze formules nog een extra term toe: $+ \epsilon$. Deze $\epsilon$ stelt een (random) getal voor dat zo gekozen wordt zodat de formule precies klopt. Deze extra term wordt ook wel de _errorterm_ genoemd, omdat dit getal de fout is tussen de voorspelling (= uitkomst van de structuurfunctie) en de echte waarde. De juiste schrijfwijze van bijvoorbeeld formule (1) is dus:

$$
y = f(\mathbf{x})  + \epsilon \tag{5}
$$

Veel beginnelingen denken dat ze een algoritme moeten kiezen met een gecompliceerde structuurfunctie en veel parameters. Want een simpele structuurfunctie, zoals (3), kan alleen maar simpele relaties modelleren. In dit geval rechte lijnen. Terwijl een ingewikkelder structuurfunctie (bijvoorbeeld met extra termen zoals $+ c_{n+1}x_1^2$ niet alleen rechte lijnen kan modelleren, maar ook kromme lijnen. In dit geval parabolen. Maar dat is niet altijd waar. Dit is de kern van dit notebook. Dus werk snel verder. Maar eerst doen we een opdracht.

<div style="background-color: lightyellow">


# Opdracht
Neem de wijn data set. Splits de data set in een train set en test set (30% van de observaties komt in de test set). Voorspel de targetvariabele 'points' op basis van 'price' en 'alcohol' waarbij je lineaire regressie gebruikt.
1. Wat is de structuurfunctie nadat je getraind hebt? Geeft de waarden van $c_0$, $c_{price}$ en $c_{alcohol}$.
2. Berekenen voor alle wijnen van de test set wat de waarde is van $\epsilon$. Maak een histogram van de waarden van $\epsilon$.

</div>ht


## Het tweede element dat vaak voorkomt: de loss functie
Naast een structuurfunctie, werken de meeste supervised learning algoritmen ook met een loss-functie.
Deze is al uitgebreid behandeld. Voor een continue target is de meest voorkomende loss functie de mean squared error loss functie:

$$
MSE = \frac{1}{n}\sum_{i = 1}^n (y_i - f(\mathbf{x_i}))^2
$$

Zoals jullie weten kun je de loss functie zowel berekenen over je train set om te kijken of het leren goed gaat, maar ook berekenen op je test set om te kijken of je niet aan het overfitten bent.

Voor een nominale target bestaan andere loss functies, zoals de cross entropy loss. 

We sommen nu even op wat de drie elementen zijn die de meeste supervised learning algoritmen in één of andere vorm hebben:
1. structuurfunctie;
2. loss functie;
3. optimalisatie algoritme.

Over het optimalisatie algoritme hebben we het niet gehad, omdat het geen rol speelt voor de Bias-Variance trade-off. Maar voor de volledigheid noemen we twee veel voorkomende optimalisatie algoritmen: Gradient Descent en Newton-Raphson, maar er bestaan er meer. Als je als beginnende data scientist voor de algoritmen die je gebruikt een idee hebt van de structuurfunctie, de gebruikte loss functie en het toegepaste optimalisatie algoritme, dan heb je al een heel goed gevoel waar je mee bezig bent!

Voordat we verder gaan met de Bias - Variance trade-off, eerst nog een korte oefening met de loss functie.


<div style="background-color: lightyellow">


# Opdracht
Ga verder met de wijn data set. Berekenen de MSE op je test set.
</div>

## Bias-Variance tradeoff in een voorbeeld
Nu we weer scherp hebben hoe we de loss kunnen uitrekenen op de test set, kunnen we teruggaan naar het eigenlijke onderwerp van dit notebook. De bias-variance tradeoff. Zoals bovenaan in dit notebook is uitgelegd, is de bias een oorzaak van een deel van de loss op de test set en de variance ook. Om hier een beter beeld van te krijgen, gaan we hier nu zelf mee rekenen.

In de praktijk kun je niet de bias en de variance in een getal uitdrukken, omdat je niet weet wat het precieze, echte, onderliggende relatie is tussen $y$ en $\mathbf{x}$. Maar voor dit voorbeeld, bedenken we zelf van een relatie tussen $y$ en $\mathbf{x}$. Dan kan het wel. Vervolgens gaan we deze relatie schatten met een drietal modellen, van simpel tot wat complexer. Als we meerdere keren een train set genereren en steeds opnieuw een model trainen, dan zullen we zien dat de uitkomst niet steeds hetzelfde is. We zullen zien dat voor eenvoudige modellen het niet zoveel verschil uitmaakt als we opnieuw trainen, maar voor ingewikkelder modellen wel. In ons voorbeeld kunnen we dan zelfs een getal hangen aan bias en variance. Op die manier kun je zelf zien wat er gebeurt als je een complexer model neemt.

Om te beginnen willen het begrip _bias_ kwantificeren. In woorden hebben we gezegd dat de bias een afwijking in de voorspelling is, veroorzaakt doordat het gekozen model te eenvoudig is om het werkelijke verband tussen $y$ en $\mathbf{x}$ te omschrijven. Als we dit willen uitdrukken in een getal, dan kan dat eigenlijk alleen door erg ingewikkelde wiskundige berekeningen uit te voeren met kansverdelingen. We zullen daarom niet de bias uitrekenen, maar een goede benadering van de bias, de _sample bias_. Deze is goed te berekenen als je een aantal keer (bijvoorbeeld 10) achter elkaar hetzelfde model fit, maar steeds op een nieuwe train set. Als we dan het gemiddelde van die voorspellingen nemen en vergelijken met de echte waarde, krijgen we de sample bias. In formulevorm:

$$
Sample Bias ^2 = \frac{1}{n} \sum_{i=1}^n  (\widehat{f(\mathbf{x_i})} - y)^2
$$

Hier is $\widehat{f(\mathbf{x_i})}$ het gemiddelde van die 10 keer trainen op steeds een andere train set. We berekenen het kwadraat omdat er straks een leuk verband blijkt te bestaan tussen de MSE op de test set, de bias in het kwadraat en de variance. Als je niet het kwadraat wilt, dan neem je gewoon de wortel van het gevonden getal.

Ook kunnen we de _sample variance_ kwantificeren. In principe is dit de mate waarin de voorspellingen verschillen als je je model traint op een nieuwe train set. In formule vorm:

$$
Sample Variance =  \frac{1}{m} \sum_{j=1}^m \frac{1}{n} \sum_{i=1}^n  (f^j(\mathbf{x_i}) - \widehat{f(\mathbf{x_i})})^2
$$
Hierbij is $m$ het aantal keer dat je je model hertraint en $f^j$ is de voorspelling van het model dat de j-de keer getraind is.

Het begint nu allemaal wel moeilijk te worden. Laten we dus snel gaan kijken naar het voorbeeld! We beginnen met het definiëren van een zelf bedachte relatie tussen $\mathbf{x}$ en $y$:

In [4]:
# Definieer de "ware" functie waarmee we de data genereren.
# x is een 7-dimensionale vector en we kiezen een combinatie van lineaire, kwadratische en niet-lineaire (sinus, log) termen.
def true_function(X):
    # X heeft vorm (n_samples, 7)
    return (1 
            + 2 * X[:, 0]               # lineair effect van feature 1
            - 1 * X[:, 1]               # lineair effect van feature 2
            + 0.5 * (X[:, 2] ** 2)        # kwadratisch effect van feature 3
            + np.sin(X[:, 3])           # niet-lineair effect van feature 4
            + np.log(1 + np.abs(X[:, 4]))  # niet-lineair effect van feature 5
            
            + 0.3 * X[:, 5] * X[:, 6])   # interactieterm tussen feature 6 en 7

Nu genereren we test data met behulp van deze relatie tussen $y$ en $\mathbf{x}$. We generen de echte waarden van $y$ ('y_true') en voegen ook een irreducibele fout toe, net als in de praktijk het geval is. Deze target met ruis noemen we 'y_test'. De x-waarden noemen we 'X_test'.

In [5]:
# Stel een vaste seed in voor reproduceerbaarheid
np.random.seed(42)

# Functie om data te genereren.
def generate_data(n_samples=50, noise_std=1.0):
    # Genereer 7 features, elk uniform verdeeld tussen -5 en 5.
    X = np.random.uniform(-5, 5, size=(n_samples, 7))
    # Bereken de "ware" y-waarden (zonder ruis)
    y_true = true_function(X)
    # Voeg normale ruis toe (voor de trainingsdata)
    noise = np.random.normal(0, noise_std, size=n_samples)
    y = y_true + noise
    return X, y, y_true

# Genereer een testset (groot genoeg en zonder ruis, zodat we de ware functie kennen)
X_test, y_test, y_test_true = generate_data(n_samples=1000, noise_std=1.0)

Nu gaan we een simpel lineaire regressie model trainen om $y$ te voorspellen, enkel op basis van de eerste kolom van de data X:

In [6]:
# We herhalen het experiment meerdere keren (bijvoorbeeld 10) met kleine trainingssets
n_experiments = 10

# Lijsten om de voorspellingen van elk model op te slaan (per experiment)
predictions_model1 = []  # Model 1: gebruikt alleen feature 1

for i in range(n_experiments):
    # Voor de trainingsset kiezen we bewust een klein aantal datapunten (bijvoorbeeld 30)
    X_train, y_train, _ = generate_data(n_samples=30, noise_std=1.0)
    
    # --- Model 1: Simpel model (alleen de eerste feature) ---
    model1 = LinearRegression()
    model1.fit(X_train[:, 0].reshape(-1, 1), y_train)
    pred1 = model1.predict(X_test[:, 0].reshape(-1, 1))
    predictions_model1.append(pred1)

# Zet de lijst met voorspellingen om in een numpy-array voor verdere berekeningen.
# De arrays hebben nu de vorm (n_experiments, n_test_samples)
predictions_model1 = np.array(predictions_model1)

# Functie om per testpunt de MSE, bias² en variantie te berekenen
def compute_bias_variance(predictions, y_true):
    # Gemiddelde voorspelling per testpunt over alle experimenten
    mean_preds = np.mean(predictions, axis=0)
    # Varianties per testpunt
    variance = np.var(predictions, axis=0)
    # Bias² per testpunt: (gemiddelde voorspelling - ware waarde)²
    bias_squared = (mean_preds - y_true) ** 2
    # MSE per testpunt: gemiddelde van (voorspelling - ware waarde)²
    mse = np.mean((predictions - y_true) ** 2, axis=0)
    return mse, bias_squared, variance

# Bereken de MSE, bias² en variantie voor het model
mse1, bias_sq1, var1 = compute_bias_variance(predictions_model1, y_test_true)


# Toon de gemiddelde MSE, bias² en variantie over de gehele testset voor het model
print("Model 1 - Simpel model (alleen feature 1):")
print("Gemiddelde MSE:", np.mean(mse1))
print("Gemiddelde bias^2:", np.mean(bias_sq1))
print("Gemiddelde variantie:", np.mean(var1))

Model 1 - Simpel model (alleen feature 1):
Gemiddelde MSE: 32.31551778490702
Gemiddelde bias^2: 30.373946343566
Gemiddelde variantie: 1.9415714413410297


Je ziet dat het grootste deel van de MSE van de test set veroorzaakt wordt doordat het model eigenlijk te simpel is. De bias is groot en de variance is klein. Daarom nu de opdracht aan jullie om de volgende twee andere modellen op dezelfde wijze te trainen en te zien wat er dan uitkomt voor de MSE, de bias en de variance.

<div style="background-color: lightyellow">


# Opdracht
1. Pas de code hierboven aan voor een lienaire regressie model dat alle features gebruikt.
2. Voeg nu aan de code hierboven nog een model toe, namelijk een polynomiale regressie (graad 2) met alle features
3. Wat gebeurt er met de MSE. En met de bias en variance? Kun je dit verklaren?
</div>

## Uitwerking

In [5]:


# We herhalen het experiment meerdere keren (bijvoorbeeld 10) met kleine trainingssets
n_experiments = 10

# Lijsten om de voorspellingen van elk model op te slaan (per experiment)
predictions_model1 = []  # Model 1: gebruikt alleen feature 1
predictions_model2 = []  # Model 2: lineaire regressie met alle features
predictions_model3 = []  # Model 3: polynomiale regressie (graad 2) met alle features

for i in range(n_experiments):
    # Voor de trainingsset kiezen we bewust een klein aantal datapunten (bijvoorbeeld 30)
    X_train, y_train, _ = generate_data(n_samples=30, noise_std=1.0)
    
    # --- Model 1: Simpel model (alleen de eerste feature) ---
    model1 = LinearRegression()
    model1.fit(X_train[:, 0].reshape(-1, 1), y_train)
    pred1 = model1.predict(X_test[:, 0].reshape(-1, 1))
    predictions_model1.append(pred1)
    
    # --- Model 2: Lineaire regressie met alle 7 features ---
    model2 = LinearRegression()
    model2.fit(X_train, y_train)
    pred2 = model2.predict(X_test)
    predictions_model2.append(pred2)
    
    # --- Model 3: Polynomiale regressie (graad 2) met alle features ---
    model3 = make_pipeline(PolynomialFeatures(degree=2, include_bias=False), LinearRegression())
    model3.fit(X_train, y_train)
    pred3 = model3.predict(X_test)
    predictions_model3.append(pred3)

# Zet de lijst met voorspellingen om in een numpy-array voor verdere berekeningen.
# De arrays hebben nu de vorm (n_experiments, n_test_samples)
predictions_model1 = np.array(predictions_model1)
predictions_model2 = np.array(predictions_model2)
predictions_model3 = np.array(predictions_model3)

# Functie om per testpunt de MSE, bias² en variantie te berekenen
def compute_bias_variance(predictions, y_true):
    # Gemiddelde voorspelling per testpunt over alle experimenten
    mean_preds = np.mean(predictions, axis=0)
    # Varianties per testpunt
    variance = np.var(predictions, axis=0)
    # Bias² per testpunt: (gemiddelde voorspelling - ware waarde)²
    bias_squared = (mean_preds - y_true) ** 2
    # MSE per testpunt: gemiddelde van (voorspelling - ware waarde)²
    mse = np.mean((predictions - y_true) ** 2, axis=0)
    return mse, bias_squared, variance

# Bereken de MSE, bias² en variantie voor elk model
mse1, bias_sq1, var1 = compute_bias_variance(predictions_model1, y_test_true)
mse2, bias_sq2, var2 = compute_bias_variance(predictions_model2, y_test_true)
mse3, bias_sq3, var3 = compute_bias_variance(predictions_model3, y_test_true)

# Toon de gemiddelde MSE, bias² en variantie over de gehele testset per model
print("Model 1 - Simpel model (alleen feature 1):")
print("Gemiddelde MSE:", np.mean(mse1))
print("Gemiddelde bias^2:", np.mean(bias_sq1))
print("Gemiddelde variantie:", np.mean(var1))
print("\nModel 2 - Lineaire regressie (alle features):")
print("Gemiddelde MSE:", np.mean(mse2))
print("Gemiddelde bias^2:", np.mean(bias_sq2))
print("Gemiddelde variantie:", np.mean(var2))
print("\nModel 3 - Polynomiale regressie (graad 2):")
print("Gemiddelde MSE:", np.mean(mse3))
print("Gemiddelde bias^2:", np.mean(bias_sq3))
print("Gemiddelde variantie:", np.mean(var3))


Model 1 - Simpel model (alleen feature 1):
Gemiddelde MSE: 32.31551778490702
Gemiddelde bias^2: 30.373946343566
Gemiddelde variantie: 1.9415714413410297

Model 2 - Lineaire regressie (alle features):
Gemiddelde MSE: 29.26822589385843
Gemiddelde bias^2: 22.231620155439145
Gemiddelde variantie: 7.036605738419279

Model 3 - Polynomiale regressie (graad 2):
Gemiddelde MSE: 73.37615727893854
Gemiddelde bias^2: 16.743715120660294
Gemiddelde variantie: 56.632442158278245
